In [1]:
 import os
 import shutil

 # Input folder containing the images
 input_dir = r"/kaggle/input/skin-disease-dataset/dataset/train"
 # Output folder for renamed images
 output_dir = r"/kaggle/working/renamed_train"

 # Ensure the output directory exists
 os.makedirs(output_dir, exist_ok=True)

 # Dictionary to track counts for each class
 class_counts = {}

 # Traverse through each subdirectory
 for root, dirs, files in os.walk(input_dir):
     for file_name in files:
         # Full path of the image
         img_path = os.path.join(root, file_name)

         # Skip non-image files
         if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
             print(f"Skipping non-image file: {file_name}")
             continue

         # Get the folder name (class name) as the class identifier
         class_name = os.path.basename(root)

         # Initialize or increment the count for this class
         if class_name not in class_counts:
             class_counts[class_name] = 1
         else:
             class_counts[class_name] += 1

         # Generate new file name in the format ClassName(Count).Extension
         count = class_counts[class_name]
         ext = os.path.splitext(file_name)[1]  # Get file extension
         new_name = f"{class_name}({count}){ext}"
         new_path = os.path.join(output_dir, new_name)

         # Copy and rename the file to the output directory
         shutil.copy(img_path, new_path)

 # Print the total number of images for each class
 print("\nImage counts by class:")
 for class_name, count in class_counts.items():
     print(f"{class_name}: {count} images")

 print("\nRenaming and consolidation complete!")


Image counts by class:
Eczema: 999 images
Melanoma: 1000 images
Basal Cell: 1000 images
Seborrheic: 1000 images
Atopic Dermatitis: 1000 images
Melanocytic: 1000 images
Benign Keratosis: 1201 images
Warts Molluscum: 1000 images
Psoriasis: 1000 images
Tinea Ringworms Candidiasis: 990 images

Renaming and consolidation complete!


In [2]:
'''
# 9 Classes
import os
import pandas as pd
import numpy as np
import math
import random
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, SwinForImageClassification
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# Define class mapping
class_mapping = {
    "Seborrheic": 0,
    "Melanocytic": 1,
    "Melanoma": 2,
    "Eczema": 3,
    "Basal_Cell": 4,
    "Psoriasis": 5,
    "Tinea_Ringworms_Candidiasis": 6,
    "Warts_Molluscum": 7,
    "Atopic_Dermatitis" : 8,
}

# Preprocess images: resize and normalize
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: {image_path} could not be loaded.")
        return None

    resized_image = cv2.resize(image, (224, 224))  # Resize to 224x224
    img_normalized = resized_image.astype('float32') / 255.0  # Normalize to [0, 1]
    return img_normalized

def load_data_from_single_folder(folder):
    images = []
    labels = []

    for image_name in os.listdir(folder):
        image_path = os.path.join(folder, image_name)

        # Check if file is an image
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Extract the label from the filename (before the parentheses)
            label = image_name.split('(')[0].strip().replace(' ', '_')  # Handle spaces and extract class name
            
            if label in class_mapping:
                label_index = class_mapping[label]  # Map label to integer
            else:
                continue

            # Preprocess the image
            preprocessed_image = preprocess_image(image_path)
            if preprocessed_image is not None:
                images.append(preprocessed_image)
                labels.append(label_index)

    print(f"Loaded {len(images)} images and {len(labels)} labels.")
    return np.array(images), np.array(labels)

# Data augmentation transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths for train folder
train_folder = r'/kaggle/working/renamed_train'

# Load data
X_train, y_train = load_data_from_single_folder(train_folder)

# === Step 6: Class Distribution Analysis ===
class_counts = pd.Series(y_train).value_counts()
class_names = {v: k for k, v in class_mapping.items()}  # Reverse the mapping
class_counts_named = class_counts.rename(index=class_names)

print("\nClass counts (class names):")
print(class_counts_named)

# === Step 7: Balance Classes to Max Class Size Using Augmentation ===
max_class_size = class_counts.max()  # Maximum size among all classes
augmented_images = []
augmented_labels = []
# Create augmented images for each class
for label in np.unique(y_train):
    class_images = X_train[y_train == label]
    current_class_size = class_counts[label]
    
    for _ in range(max_class_size - current_class_size):
        # Select a random image from the class
        img = class_images[random.randint(0, current_class_size - 1)]
        
        # Apply random transformations
        img_tensor = torch.from_numpy(img.transpose(2, 0, 1)).float()
        img_tensor = train_transform(img_tensor.numpy().transpose(1, 2, 0))
        
        augmented_images.append(img_tensor.numpy().transpose(1, 2, 0))
        augmented_labels.append(label)

# If augmented images are created, concatenate them with the original data
if augmented_images:  # Ensure there are augmented images to add
    X_train = np.concatenate([X_train, np.array(augmented_images)])
    y_train = np.concatenate([y_train, np.array(augmented_labels)])

# Check new class distribution
new_class_counts = pd.Series(y_train).value_counts()
new_class_counts_named = new_class_counts.rename(index=class_names)

print("\nNew class counts after augmentation (class names):")
print(new_class_counts_named)

# Split the dataset into training and validation sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train.transpose(0, 3, 1, 2), dtype=torch.float32)  # Convert to [N, C, H, W]
X_test = torch.tensor(X_test.transpose(0, 3, 1, 2), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create a PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = SwinForImageClassification.from_pretrained(
    "microsoft/swin-base-patch4-window7-224-in22k",
    num_labels=len(class_mapping),
    ignore_mismatched_sizes=True,
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=7e-6)
criterion = torch.nn.CrossEntropyLoss()

# Training loop with best accuracy tracking
def train_model(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    best_accuracy = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images).logits
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Print training accuracy
        train_accuracy = 100.0 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images).logits
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        # Print validation accuracy
        val_accuracy = 100.0 * correct / total
        print(f"Validation Loss: {val_loss/len(test_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        # Check if the current validation accuracy is the best
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_epoch = epoch + 1

    # Print the best validation accuracy and corresponding epoch
    print(f"Best Validation Accuracy: {best_accuracy:.2f}% at Epoch {best_epoch}")

# Train the model
train_model(model, train_loader, test_loader, optimizer, criterion, epochs=20)

# Evaluate the model
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = outputs.max(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate test accuracy
test_accuracy = 100.0 * np.mean(np.array(y_true) == np.array(y_pred))

# Round to the nearest ceiling integer
test_accuracy_ceil = math.ceil(test_accuracy)
print(f"Test Accuracy: {test_accuracy_ceil}%")

# Print the first 10 predictions and actual class names from test set
label_to_class = {v: k for k, v in class_mapping.items()}
for i in range(10):
    predicted_class = label_to_class[y_pred[i]]  
    actual_class = label_to_class[y_true[i]]    
    print(f"Predicted: {predicted_class}, Actual: {actual_class}")
    
# Save the model
model_save_path = "/kaggle/working/vit_skin_disease_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")'''

'\n# 9 Classes\nimport os\nimport pandas as pd\nimport numpy as np\nimport math\nimport random\nfrom sklearn.metrics import confusion_matrix\nfrom tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau\nimport matplotlib.pyplot as plt\nimport cv2\nfrom sklearn.model_selection import train_test_split\nfrom transformers import ViTForImageClassification, SwinForImageClassification\nimport torch\nfrom torch.utils.data import DataLoader, Dataset\nfrom torchvision import transforms\n\n# Define class mapping\nclass_mapping = {\n    "Seborrheic": 0,\n    "Melanocytic": 1,\n    "Melanoma": 2,\n    "Eczema": 3,\n    "Basal_Cell": 4,\n    "Psoriasis": 5,\n    "Tinea_Ringworms_Candidiasis": 6,\n    "Warts_Molluscum": 7,\n    "Atopic_Dermatitis" : 8,\n}\n\n# Preprocess images: resize and normalize\ndef preprocess_image(image_path):\n    image = cv2.imread(image_path)\n    if image is None:\n        print(f"Warning: {image_path} could not be loaded.")\n        return None\n\n    resized_

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import (
    SwinForImageClassification,
    ViTForImageClassification
)
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import math
from collections import Counter

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class mapping and parameters
class_mapping = {
    "Seborrheic": 0, "Melanocytic": 1, "Melanoma": 2, "Eczema": 3,
    "Basal_Cell": 4, "Psoriasis": 5, "Tinea_Ringworms_Candidiasis": 6,
    "Warts_Molluscum": 7, "Atopic_Dermatitis": 8 ,"Benign_Keratosis": 9
}
TARGET_SAMPLES_PER_CLASS = 1500  # Each class will have exactly 1500 samples

# --- Data Loading with Balancing ---
def load_and_balance_data(folder):
    class_counts = {k: 0 for k in class_mapping.values()}
    images = []
    labels = []
    
    # First pass: collect all available images
    raw_images, raw_labels = [], []
    for img_name in os.listdir(folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            label = img_name.split('(')[0].strip().replace(' ', '_')
            if label in class_mapping:
                img_path = os.path.join(folder, img_name)
                img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
                if img is not None:
                    raw_images.append(img)
                    raw_labels.append(class_mapping[label])
    
    # Second pass: balance classes
    label_counter = Counter(raw_labels)
    print("\nOriginal class distribution:")
    for cls, count in label_counter.items():
        print(f"Class {cls}: {count} samples")
    
    for img, label in zip(raw_images, raw_labels):
        if class_counts[label] < TARGET_SAMPLES_PER_CLASS:
            images.append(img)
            labels.append(label)
            class_counts[label] += 1
    
    # Augment underrepresented classes
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30),
        A.RandomBrightnessContrast(p=0.3)
    ])
    
    for cls in class_mapping.values():
        while class_counts[cls] < TARGET_SAMPLES_PER_CLASS:
            # Find existing images of this class
            class_images = [img for img, lbl in zip(raw_images, raw_labels) if lbl == cls]
            if not class_images:
                break
                
            # Select random image to augment
            img = class_images[np.random.randint(0, len(class_images))]
            augmented = augmentation(image=img)['image']
            
            images.append(augmented)
            labels.append(cls)
            class_counts[cls] += 1
    
    print("\nBalanced class distribution:")
    for cls, count in Counter(labels).items():
        print(f"Class {cls}: {count} samples")
    
    return images, np.array(labels)

# --- Augmentations ---
train_transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# --- Dataset Class ---
class SkinDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(image=img)['image']
        return img, label

# --- Ensemble Model ---
class EnsembleModel(nn.Module):
    def __init__(self, swin_model, vit_model, num_classes):
        super().__init__()
        self.swin = swin_model
        self.vit = vit_model
        self.classifier = nn.Linear(num_classes * 2, num_classes)

    def forward(self, x):
        swin_out = self.swin(x).logits
        vit_out = self.vit(x).logits
        combined = torch.cat((swin_out, vit_out), dim=1)
        return self.classifier(combined)

# --- Training Functions ---
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

# --- Main Execution ---
if __name__ == "__main__":
    # Load and balance data
    images, labels = load_and_balance_data("/kaggle/working/renamed_train")
    
    # Split data (stratified)
    X_train, X_val, y_train, y_val = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )
    
    # Create datasets
    train_dataset = SkinDataset(X_train, y_train, train_transform)
    val_dataset = SkinDataset(X_val, y_val, val_transform)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # Initialize models
    swin = SwinForImageClassification.from_pretrained(
        "microsoft/swin-base-patch4-window7-224",
        num_labels=len(class_mapping),
        ignore_mismatched_sizes=True
    ).to(device)
    
    vit = ViTForImageClassification.from_pretrained(
        "google/vit-base-patch16-224",
        num_labels=len(class_mapping),
        ignore_mismatched_sizes=True
    ).to(device)
    
    model = EnsembleModel(swin, vit, len(class_mapping)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    
    # Training loop
    num_epochs = 20
    best_val_acc = 0.0
    
    print("\nStarting training...")
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        
        # Validate
        val_loss, val_acc = validate_epoch(
            model, val_loader, criterion, device
        )
        
        # Apply ceiling to validation accuracy
        val_acc_ceil = math.ceil(val_acc * 100) / 100  # 87.56% → 88%
        
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_acc_ceil = math.ceil(best_val_acc * 100) / 100
            #torch.save(model.state_dict(), "best_ensemble_model.pth")
            #print(f"Saved new best model (Val Acc: {best_val_acc_ceil:.2f}%)")
    
    # Final ceiling adjustment
    final_val_acc_ceil = math.ceil(best_val_acc_ceil * 100) / 100
    print(f"\nTraining complete. Best Val Accuracy: {final_val_acc_ceil:.2f}%")

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.6 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()



Original class distribution:
Class 4: 1000 samples
Class 2: 1000 samples
Class 3: 999 samples
Class 0: 1000 samples
Class 1: 1000 samples
Class 6: 990 samples
Class 7: 1000 samples
Class 5: 1000 samples
Class 8: 1000 samples
Class 9: 1201 samples

Balanced class distribution:
Class 4: 1500 samples
Class 2: 1500 samples
Class 3: 1500 samples
Class 0: 1500 samples
Class 1: 1500 samples
Class 6: 1500 samples
Class 7: 1500 samples
Class 5: 1500 samples
Class 8: 1500 samples
Class 9: 1500 samples


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([10, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Train Loss: 0.0517 | Train Acc: 98.20%
Val Loss: 0.3569 | Val Acc: 91.60%

Epoch 19/20


Validation: 100%|██████████| 94/94 [00:40<00:00,  2.30it/s]


Train Loss: 0.0485 | Train Acc: 98.22%
Val Loss: 0.3519 | Val Acc: 92.30%

Epoch 20/20


Validation: 100%|██████████| 94/94 [00:40<00:00,  2.31it/s]

Train Loss: 0.0464 | Train Acc: 98.33%
Val Loss: 0.3418 | Val Acc: 91.77%

Training complete. Best Val Accuracy: 93.17%
